In [4]:
import numpy as np
import json
import random
import importlib
from pprint import pprint

import os
import torch
from tbsim.configs.scene_edit_config import SceneEditingConfig

In [1]:
a = {0: 'a'}
key = list(a.keys())[0]  # Convert keys() view to list and get first key
a[key]

In [7]:
key = a.keys()

In [9]:
a.key()

AttributeError: 'dict' object has no attribute 'key'

In [6]:
type(list(a.keys())[0])

int

In [5]:
# args & configs
cfg = SceneEditingConfig()
config_file = '/local-scratch2/lufan/traceMR/configs/eval/orca/target_pos.json'
eval_class = 'Diffuser'
policy_ckpt_dir = '/local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed'
policy_ckpt_key = 'iter40000'
results_root_dir = '/local-scratch2/lufan/traceMR/out/orca_mixed_out'
num_scenes_per_batch = 1
render = False
render_img = True
render_size = 400
render_px_per_m = 12.0
seed = None
prefix = None
ground_truth = False
editing_source = 'heuristic'

if config_file is not None:
    external_cfg = json.load(open(config_file, "r"))
    cfg.update(**external_cfg)

if eval_class is not None:
    cfg.eval_class = eval_class

if policy_ckpt_dir is not None:
    assert policy_ckpt_key is not None, "Please specify a key to look for the checkpoint, e.g., 'iter50000'"
    cfg.ckpt.policy.ckpt_dir = policy_ckpt_dir
    cfg.ckpt.policy.ckpt_key = policy_ckpt_key

if num_scenes_per_batch is not None:
    cfg.num_scenes_per_batch = num_scenes_per_batch

if cfg.name is None:
    cfg.name = cfg.eval_class

if prefix is not None:
    cfg.name = prefix + cfg.name

if seed is not None:
    cfg.seed = seed
if results_root_dir is not None:
    cfg.results_dir = os.path.join(results_root_dir, cfg.name)
else:
    cfg.results_dir = os.path.join(cfg.results_dir, cfg.name)
assert cfg.env == "trajdata", "trajdata is the only supported environment"
if editing_source is not None:
    cfg.edits.editing_source = editing_source
if not isinstance(cfg.edits.editing_source, list):
    cfg.edits.editing_source = [cfg.edits.editing_source]

cfg.experience_hdf5_path = os.path.join(cfg.results_dir, "data.hdf5")

for k in cfg[cfg.env]:  # copy env-specific config to the global-level
    cfg[k] = cfg[cfg.env][k]
cfg.pop("trajdata")

render_cfg = {
    'size' : render_size,
    'px_per_m' : render_px_per_m,
}

cfg.lock()

In [4]:
# basic setup, before init policy
from tbsim.utils.trajdata_utils import set_global_trajdata_batch_env
"""
run_scene_editor(
    cfg,
    save_cfg=True,
    data_to_disk=True,
    render_to_video=args.render,
    render_to_img=args.render_img,
    render_cfg=render_cfg,
    use_gt=args.ground_truth,
)
"""
eval_cfg = cfg
save_cfg = True
data_to_disk = True
render_to_video = render
render_to_img = render_img
render_cfg = render_cfg
use_gt = ground_truth

set_global_trajdata_batch_env(eval_cfg.trajdata_source_test[0])
print(eval_cfg)
# for reproducibility
np.random.seed(eval_cfg.seed)
random.seed(eval_cfg.seed)
torch.manual_seed(eval_cfg.seed)
torch.cuda.manual_seed(eval_cfg.seed)

# basic setup
print('saving results to {}'.format(eval_cfg.results_dir))
os.makedirs(eval_cfg.results_dir, exist_ok=True)
if render_to_video:
    os.makedirs(os.path.join(eval_cfg.results_dir, "videos/"), exist_ok=True)
if render_to_video or render_to_img:
    os.makedirs(os.path.join(eval_cfg.results_dir, "viz/"), exist_ok=True)
if save_cfg:
    json.dump(eval_cfg, open(os.path.join(eval_cfg.results_dir, "config.json"), "w+"))
if data_to_disk and os.path.exists(eval_cfg.experience_hdf5_path):
    os.remove(eval_cfg.experience_hdf5_path)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


{
    "name": "orca_map_open_loop_target_pos",
    "env": "trajdata",
    "dataset_path": null,
    "eval_class": "Diffuser",
    "seed": 0,
    "num_scenes_per_batch": 1,
    "num_scenes_to_evaluate": 100,
    "num_episode_repeats": 1,
    "start_frame_index_each_episode": null,
    "seed_each_episode": null,
    "ego_only": false,
    "ckpt_root_dir": "checkpoints/",
    "experience_hdf5_path": "/local-scratch2/lufan/traceMR/out/orca_mixed_out/orca_map_open_loop_target_pos/data.hdf5",
    "results_dir": "/local-scratch2/lufan/traceMR/out/orca_mixed_out/orca_map_open_loop_target_pos",
    "ckpt": {
        "policy": {
            "ckpt_dir": "/local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed",
            "ckpt_key": "iter40000",
            "ngc_job_id": null
        }
    },
    "policy": {
        "num_action_samples": 20,
        "guide_as_filter_only": false,
        "guide_with_gt": false,
        "class_free_guide_w": 0.0,
        "guide_clean": true
    },
    "metrics": {
 

In [5]:
policy_composers = importlib.import_module("tbsim.evaluation.policy_composers")
composer_class = getattr(policy_composers, eval_cfg.eval_class)

In [6]:
# create policy and rollout wrapper
policy_composers = importlib.import_module("tbsim.evaluation.policy_composers")
composer_class = getattr(policy_composers, eval_cfg.eval_class)
composer = composer_class(eval_cfg, device)
policy, exp_config = composer.get_policy()
policy_model = policy.model

# if use_gt is ignored here

Checkpoint path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/iter40000.ckpt
Config path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/config.json


DIFFUSER: Dropping map input conditioning with p = 0.100000 during training...
DIFFUSER: Dropping neighbor traj input conditioning with p = 0.100000 during training...
[ models/temporal ] Channel dimensions: [(38, 64), (64, 128), (128, 256)]
DIFFUSER: using EMA... val and get_action will use ema model


In [11]:
# determines cfg for rasterizing agents
def set_global_trajdata_batch_raster_cfg(raster_cfg):
    global BATCH_RASTER_CFG
    assert "include_hist" in raster_cfg
    assert "pixel_size" in raster_cfg
    assert "raster_size" in raster_cfg
    assert "ego_center" in raster_cfg
    assert "num_sem_layers" in raster_cfg
    assert "no_map_fill_value" in raster_cfg
    assert "drivable_layers" in raster_cfg
    BATCH_RASTER_CFG = raster_cfg

set_global_trajdata_batch_raster_cfg(exp_config.env.rasterizer)

In [8]:
# control the full scene with same policy
from tbsim.policies.common import Action, Plan, RolloutAction
class RolloutWrapper(object):
    """A wrapper policy that can (optionally) control both ego and other agents in a scene"""

    def __init__(self, ego_policy=None, agents_policy=None, pass_agent_obs=True):
        self.device = ego_policy.device if agents_policy is None else agents_policy.device
        self.ego_policy = ego_policy
        self.agents_policy = agents_policy
        self.pass_agent_obs = pass_agent_obs

    def eval(self):
        self.ego_policy.eval()
        self.agents_policy.eval()

    def get_action(self, obs, step_index) -> RolloutAction:
        ego_action = None
        ego_action_info = None
        agents_action = None
        agents_action_info = None
        if self.ego_policy is not None:
            assert obs["ego"] is not None
            with torch.no_grad():
                if self.pass_agent_obs:
                    ego_action, ego_action_info = self.ego_policy.get_action(
                        obs["ego"], step_index = step_index,agent_obs = obs["agents"])
                else:
                    ego_action, ego_action_info = self.ego_policy.get_action(
                        obs["ego"], step_index = step_index)
        if self.agents_policy is not None:
            assert obs["agents"] is not None
            with torch.no_grad():
                agents_action, agents_action_info = self.agents_policy.get_action(
                    obs["agents"], step_index = step_index)
        return RolloutAction(ego_action, ego_action_info, agents_action, agents_action_info)

rollout_policy = RolloutWrapper(agents_policy=policy)

In [9]:
print(exp_config.algo)

{
    "name": "diffuser",
    "eval_class": "Diffuser",
    "map_encoder_model_arch": "resnet18",
    "diffuser_model_arch": "TemporalMapUnet",
    "rasterized_map": true,
    "use_map_feat_global": false,
    "use_map_feat_grid": true,
    "base_dim": 32,
    "horizon": 52,
    "n_diffusion_steps": 100,
    "action_weight": 1,
    "loss_discount": 1,
    "dim_mults": [
        2,
        4,
        8
    ],
    "loss_type": "l2",
    "use_ema": true,
    "ema_step": 1,
    "ema_decay": 0.995,
    "ema_start_step": 2000,
    "diffuser_input_mode": "state_and_action",
    "conditioning_drop_map_p": 0.1,
    "conditioning_drop_neighbor_p": 0.1,
    "conditioning_drop_fill": 0.5,
    "cond_feat_dim": 256,
    "map_feature_dim": 256,
    "map_grid_feature_dim": 32,
    "history_feature_dim": 128,
    "history_num_frames": 30,
    "history_num_frames_ego": 30,
    "history_num_frames_agents": 30,
    "future_num_frames": 52,
    "step_time": 0.1,
    "dynamics": {
        "type": "Unicycle"

In [14]:
# create env
from tbsim.evaluation.env_builders import EnvUnifiedBuilder
env_builder = EnvUnifiedBuilder(eval_config=eval_cfg, exp_config=exp_config, device=device)
env = env_builder.get_env()

Loading data for matched scene tags: ['test-orca_maps']


Calculating Agent Data (Serially): 100%|██████████| 100/100 [00:00<00:00, 15472.00it/s]


100 scenes in the scene index.


Structuring Agent Data Index: 100%|██████████| 100/100 [00:00<00:00, 24621.68it/s]

{
    "name": "trajdata",
    "data_generation_params": {
        "trajdata_centric": "agent",
        "trajdata_only_types": [
            "pedestrian"
        ],
        "trajdata_predict_types": [
            "pedestrian"
        ],
        "trajdata_scene_desc_contains": null,
        "trajdata_incl_map": true,
        "trajdata_max_agents_distance": 15.0,
        "trajdata_standardize_data": true
    },
    "rasterizer": {
        "include_hist": false,
        "num_sem_layers": 2,
        "drivable_layers": [
            0
        ],
        "rgb_idx_groups": [
            [
                1
            ],
            [
                0
            ],
            [
                1
            ]
        ],
        "raster_size": 224,
        "pixel_size": 0.08333333333333333,
        "ego_center": [
            -0.5,
            0.0
        ],
        "no_map_fill_value": 0.5
    },
    "simulation": {
        "num_simulation_steps": 50,
        "start_frame_index": 31
    }
}

dict

In [18]:
# set up heuristic guidance config
heuristic_config = None
if "heuristic" in eval_cfg.edits.editing_source:
    if eval_cfg.edits.heuristic_config is not None:
        heuristic_config = eval_cfg.edits.heuristic_config
    else:
        heuristic_config = []


In [19]:
# preparation for render
render_rasterizer = None
if render_to_video or render_to_img:
    from tbsim.utils.viz_utils import get_trajdata_renderer
    # initialize rasterizer once for all scenes
    render_rasterizer = get_trajdata_renderer(eval_cfg.trajdata_source_test,
                                                eval_cfg.trajdata_data_dirs,
                                                raster_size=render_cfg['size'],
                                                px_per_m=render_cfg['px_per_m'],
                                                rebuild_maps=False,
                                                cache_location=eval_cfg.trajdata_cache_location)


In [20]:
result_stats = None
scene_i = 0
eval_scenes = eval_cfg.eval_scenes

# while scene_i < eval_cfg.num_scenes_to_evaluate:
scene_indices = eval_scenes[scene_i: scene_i + eval_cfg.num_scenes_per_batch]
scene_i += eval_cfg.num_scenes_per_batch
print(scene_indices)
# check to make sure all the scenes are valid at starting step
scenes_valid = env.reset(scene_indices=scene_indices, start_frame_index=None)
scene_indices = [si for si, sval in zip(scene_indices, scenes_valid) if sval]
if len(scene_indices) == 0:
    torch.cuda.empty_cache()
    # continue

# if requested, split each scene up into multiple simulations
start_frame_index = [[exp_config.algo.history_num_frames+1]] * len(scene_indices)
if eval_cfg.num_sim_per_scene > 1:
    start_frame_index = []
    for si in range(len(scene_indices)):
        cur_scene = env._current_scenes[si].scene
        sframe = exp_config.algo.history_num_frames+1
        # want to make sure there's GT for the full rollout
        eframe = cur_scene.length_timesteps - eval_cfg.num_simulation_steps
        scene_frame_inds = np.linspace(sframe, eframe, num=eval_cfg.num_sim_per_scene, dtype=int).tolist()
        start_frame_index.append(scene_frame_inds)

print('Starting frames in current scenes:')
print(start_frame_index)


[0]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
for ei in range(eval_cfg.num_sim_per_scene):
    guidance_config = None

    cur_start_frames = [scene_start[ei] for scene_start in start_frame_index]
    # double check all scenes are valid at the current start step
    scenes_valid = env.reset(scene_indices=scene_indices, start_frame_index=cur_start_frames)
    sim_scene_indices = [si for si, sval in zip(scene_indices, scenes_valid) if sval]
    sim_start_frames = [sframe for sframe, sval in zip(cur_start_frames, scenes_valid) if sval]
    

In [37]:
heuristic_config

[{'name': 'target_pos', 'weight': 30000.0, 'params': {'target_time': 40}}]

In [36]:
# eval_cfg.edits.editing_source ['heuristic']
from tbsim.utils.scene_edit_utils import compute_heuristic_guidance


heuristic_guidance_cfg = compute_heuristic_guidance(heuristic_config,
                                                    env,
                                                    sim_scene_indices,
                                                    sim_start_frames)
if len(heuristic_config) > 0: # means that the compute_heuristic_guiance exist
    valid_scene_inds = []
    for sci, sc_cfg in enumerate(heuristic_guidance_cfg):
        if len(sc_cfg) > 0:
            valid_scene_inds.append(sci)
    
    # collect only valid scenes under the given heuristic config
    heuristic_guidance_cfg = [heuristic_guidance_cfg[vi] for vi in valid_scene_inds]
    print("what do we get from the heuristic_guidance_cfg[vi]?")
    import pdb; pdb.set_trace()
    sim_scene_indices = [sim_scene_indices[vi] for vi in valid_scene_inds]
    sim_start_frames = [sim_start_frames[vi] for vi in valid_scene_inds]
    # skip if no valid

guidance_config = merge_guidance_configs(guidance_config, heuristic_guidance_cfg)



['heuristic']

In [ ]:
stats, info = guided_rollout(env, rollout_policy, policy_model, n_step_action=eval_cfg.n_step_action,
                             guidance_config=guidance_config,
                             scene_indices=sim_scene_indices,
                             obs_to_torch=True,
                             horizon=eval_cfg.num_simulation_steps,
                             use_gt=use_gt,
                             start_frames=sim_start_frames,
                             )

In [ ]:
print(info["scene_index"])
print(sim_start_frames)
pprint(stats)

# Playing with tbsim_env
create a new env

In [16]:
import numpy as np
import json
import random
import importlib
from pprint import pprint

import os
import torch
from tbsim.configs.scene_edit_config import SceneEditingConfig

In [15]:
# args & configs
cfg = SceneEditingConfig()
config_file = '/local-scratch2/lufan/traceMR/configs/eval/orca/target_pos.json'
eval_class = 'Diffuser'
policy_ckpt_dir = '/local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed'
policy_ckpt_key = 'iter40000'
results_root_dir = '/local-scratch2/lufan/traceMR/out/orca_mixed_out'
num_scenes_per_batch = 1
render = False
render_img = True
render_size = 400
render_px_per_m = 12.0
seed = None
prefix = None
ground_truth = False
editing_source = 'heuristic'

if config_file is not None:
    external_cfg = json.load(open(config_file, "r"))
    cfg.update(**external_cfg)

if eval_class is not None:
    cfg.eval_class = eval_class

if policy_ckpt_dir is not None:
    assert policy_ckpt_key is not None, "Please specify a key to look for the checkpoint, e.g., 'iter50000'"
    cfg.ckpt.policy.ckpt_dir = policy_ckpt_dir
    cfg.ckpt.policy.ckpt_key = policy_ckpt_key

if num_scenes_per_batch is not None:
    cfg.num_scenes_per_batch = num_scenes_per_batch

if cfg.name is None:
    cfg.name = cfg.eval_class

if prefix is not None:
    cfg.name = prefix + cfg.name

if seed is not None:
    cfg.seed = seed
if results_root_dir is not None:
    cfg.results_dir = os.path.join(results_root_dir, cfg.name)
else:
    cfg.results_dir = os.path.join(cfg.results_dir, cfg.name)
assert cfg.env == "trajdata", "trajdata is the only supported environment"
if editing_source is not None:
    cfg.edits.editing_source = editing_source
if not isinstance(cfg.edits.editing_source, list):
    cfg.edits.editing_source = [cfg.edits.editing_source]

cfg.experience_hdf5_path = os.path.join(cfg.results_dir, "data.hdf5")

for k in cfg[cfg.env]:  # copy env-specific config to the global-level
    cfg[k] = cfg[cfg.env][k]
cfg.pop("trajdata")

render_cfg = {
    'size' : render_size,
    'px_per_m' : render_px_per_m,
}

cfg.lock()

In [21]:
from tbsim.evaluation.env_builders import EnvUnifiedBuilder
from tbsim.utils.trajdata_utils import set_global_trajdata_batch_raster_cfg

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
policy_composers = importlib.import_module("tbsim.evaluation.policy_composers")
composer_class = getattr(policy_composers, eval_cfg.eval_class)
composer = composer_class(eval_cfg, device)
policy, exp_config = composer.get_policy()
eval_cfg = cfg
exp_config = exp_config
device = device
env_builder = EnvUnifiedBuilder(eval_config=eval_cfg, exp_config=exp_config, device=device)
env = env_builder.get_env()
set_global_trajdata_batch_raster_cfg(exp_config.env.rasterizer)
env.reset()
obs = env.get_observation()


Checkpoint path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/iter40000.ckpt
Config path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/config.json
DIFFUSER: Dropping map input conditioning with p = 0.100000 during training...
DIFFUSER: Dropping neighbor traj input conditioning with p = 0.100000 during training...
[ models/temporal ] Channel dimensions: [(38, 64), (64, 128), (128, 256)]
DIFFUSER: using EMA... val and get_action will use ema model
Loading data for matched scene tags: ['test-orca_maps']


Calculating Agent Data (Serially): 100%|██████████| 100/100 [00:00<00:00, 61079.13it/s]


100 scenes in the scene index.


Structuring Agent Data Index: 100%|██████████| 100/100 [00:00<00:00, 64736.90it/s]

{
    "name": "trajdata",
    "data_generation_params": {
        "trajdata_centric": "agent",
        "trajdata_only_types": [
            "pedestrian"
        ],
        "trajdata_predict_types": [
            "pedestrian"
        ],
        "trajdata_scene_desc_contains": null,
        "trajdata_incl_map": true,
        "trajdata_max_agents_distance": 15.0,
        "trajdata_standardize_data": true
    },
    "rasterizer": {
        "include_hist": false,
        "num_sem_layers": 2,
        "drivable_layers": [
            0
        ],
        "rgb_idx_groups": [
            [
                1
            ],
            [
                0
            ],
            [
                1
            ]
        ],
        "raster_size": 224,
        "pixel_size": 0.08333333333333333,
        "ego_center": [
            -0.5,
            0.0
        ],
        "no_map_fill_value": 0.5
    },
    "simulation": {
        "num_simulation_steps": 50,
        "start_frame_index": 31
    }
}

In [46]:
count = 0
for i in range(200):
    env.reset()
    obs=env.get_observation()
    if 'ego' in obs.keys():
        count += 1

In [22]:
print(type(obs))
print(obs.keys())
print(obs["agents"].keys())

<class 'dict'>
dict_keys(['agents'])
dict_keys(['data_idx', 'dt', 'agent_type', 'curr_agent_state', 'agent_hist', 'agent_hist_extent', 'agent_hist_len', 'agent_fut', 'agent_fut_extent', 'agent_fut_len', 'num_neigh', 'neigh_types', 'neigh_hist', 'neigh_hist_extents', 'neigh_hist_len', 'neigh_fut', 'neigh_fut_extents', 'neigh_fut_len', 'robot_fut_len', 'maps', 'maps_resolution', 'rasters_from_world_tf', 'agents_from_world_tf', 'extras', 'image', 'drivable_map', 'target_positions', 'target_yaws', 'target_availabilities', 'history_positions', 'history_yaws', 'history_speeds', 'history_availabilities', 'curr_speed', 'centroid', 'yaw', 'type', 'extent', 'raster_from_agent', 'agent_from_raster', 'raster_from_world', 'agent_from_world', 'world_from_agent', 'all_other_agents_history_positions', 'all_other_agents_history_yaws', 'all_other_agents_history_speeds', 'all_other_agents_history_availabilities', 'all_other_agents_history_availability', 'all_other_agents_curr_speed', 'all_other_agents_fu

# Restart

## Import & Args

In [1]:
import numpy as np
import json
import random
import importlib
from pprint import pprint

import os
import torch
from tbsim.configs.scene_edit_config import SceneEditingConfig

In [4]:
# args & configs
cfg = SceneEditingConfig()
config_file = '/local-scratch2/lufan/traceMR/configs/eval/orca/target_pos.json'
eval_class = 'Diffuser'
policy_ckpt_dir = '/local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed'
policy_ckpt_key = 'iter40000'
results_root_dir = '/local-scratch2/lufan/traceMR/out/orca_mixed_out'
num_scenes_per_batch = 1
render = False
render_img = True
render_size = 400
render_px_per_m = 12.0
seed = None
prefix = None
ground_truth = False
editing_source = 'heuristic'

if config_file is not None:
    external_cfg = json.load(open(config_file, "r"))
    cfg.update(**external_cfg)

if eval_class is not None:
    cfg.eval_class = eval_class

if policy_ckpt_dir is not None:
    assert policy_ckpt_key is not None, "Please specify a key to look for the checkpoint, e.g., 'iter50000'"
    cfg.ckpt.policy.ckpt_dir = policy_ckpt_dir
    cfg.ckpt.policy.ckpt_key = policy_ckpt_key

if num_scenes_per_batch is not None:
    cfg.num_scenes_per_batch = num_scenes_per_batch

if cfg.name is None:
    cfg.name = cfg.eval_class

if prefix is not None:
    cfg.name = prefix + cfg.name

if seed is not None:
    cfg.seed = seed
if results_root_dir is not None:
    cfg.results_dir = os.path.join(results_root_dir, cfg.name)
else:
    cfg.results_dir = os.path.join(cfg.results_dir, cfg.name)
assert cfg.env == "trajdata", "trajdata is the only supported environment"
if editing_source is not None:
    cfg.edits.editing_source = editing_source
if not isinstance(cfg.edits.editing_source, list):
    cfg.edits.editing_source = [cfg.edits.editing_source]

cfg.experience_hdf5_path = os.path.join(cfg.results_dir, "data.hdf5")

for k in cfg[cfg.env]:  # copy env-specific config to the global-level
    cfg[k] = cfg[cfg.env][k]
cfg.pop("trajdata")

render_cfg = {
    'size' : render_size,
    'px_per_m' : render_px_per_m,
}

cfg.lock()

## Create Env & Basic setup
commonly used config:
"n_step_action": 50,
"num_simulation_steps": 50,
"skip_first_n": 0,
"num_sim_per_scene": 1
eval_scences: [0, ... , 100]


In [6]:
eval_cfg = cfg

In [7]:
# Basic Setup
save_cfg = True
data_to_disk = True
render_to_video = False
render_to_img = True
render_cfg = {'size': 400, 'px_per_m': 12.0}
use_gt = False

from tbsim.utils.trajdata_utils import set_global_trajdata_batch_env
set_global_trajdata_batch_env(eval_cfg.trajdata_source_test[0])
# for reproducibility, fix seed
np.random.seed(eval_cfg.seed)
random.seed(eval_cfg.seed)
torch.manual_seed(eval_cfg.seed)
torch.cuda.manual_seed(eval_cfg.seed)
# basic setup
print('saving results to {}'.format(eval_cfg.results_dir))
os.makedirs(eval_cfg.results_dir, exist_ok=True)
if render_to_video:
    os.makedirs(os.path.join(eval_cfg.results_dir, "videos/"), exist_ok=True)
if render_to_video or render_to_img:
    os.makedirs(os.path.join(eval_cfg.results_dir, "viz/"), exist_ok=True)
if save_cfg:
    json.dump(eval_cfg, open(os.path.join(eval_cfg.results_dir, "config.json"), "w+"))
if data_to_disk and os.path.exists(eval_cfg.experience_hdf5_path):
    os.remove(eval_cfg.experience_hdf5_path)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

saving results to /local-scratch2/lufan/traceMR/out/orca_mixed_out/orca_map_open_loop_target_pos


In [8]:
# create policy and rollout wrapper
policy_composers = importlib.import_module("tbsim.evaluation.policy_composers")
composer_class = getattr(policy_composers, eval_cfg.eval_class)
composer = composer_class(eval_cfg, device)
policy, exp_config = composer.get_policy()
policy_model = policy.model

Checkpoint path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/iter40000.ckpt
Config path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/config.json
DIFFUSER: Dropping map input conditioning with p = 0.100000 during training...
DIFFUSER: Dropping neighbor traj input conditioning with p = 0.100000 during training...
[ models/temporal ] Channel dimensions: [(38, 64), (64, 128), (128, 256)]
DIFFUSER: using EMA... val and get_action will use ema model


In [12]:
# determines cfg for rasterizing agents
set_global_trajdata_batch_raster_cfg(exp_config.env.rasterizer)

# control the full scene with same policy
from tbsim.policies.wrappers import RolloutWrapper
rollout_policy = RolloutWrapper(agents_policy=policy)

In [13]:
# create env
env_builder = EnvUnifiedBuilder(eval_config=eval_cfg, exp_config=exp_config, device=device)
env = env_builder.get_env()

NameError: name 'EnvUnifiedBuilder' is not defined

In [64]:
# set up heuristic guidance config
heuristic_config = None
if "heuristic" in eval_cfg.edits.editing_source:
    if eval_cfg.edits.heuristic_config is not None:
        heuristic_config = eval_cfg.edits.heuristic_config
    else:
        heuristic_config = []

In [65]:
#render related
render_rasterizer = None
if render_to_video or render_to_img:
    from tbsim.utils.viz_utils import get_trajdata_renderer
    # initialize rasterizer once for all scenes
    render_rasterizer = get_trajdata_renderer(eval_cfg.trajdata_source_test,
                                                eval_cfg.trajdata_data_dirs,
                                                raster_size=render_cfg['size'],
                                                px_per_m=render_cfg['px_per_m'],
                                                rebuild_maps=False,
                                                cache_location=eval_cfg.trajdata_cache_location)

In [73]:
# scene related preprocess
scene_i = 0
eval_scenes = eval_cfg.eval_scenes
# while scene_i < eval_cfg.num_scenes_to_evaluate
print(f'num_scenes_per_batch: {num_scenes_per_batch}')
# starting here, it's in the while scene_i < eval_cfg.num_scenes_to_evaluate loop
scene_indices = eval_scenes[scene_i: scene_i + eval_cfg.num_scenes_per_batch]
scene_i += eval_cfg.num_scenes_per_batch
print(scene_indices)

# check to make sure all the scenes are valid at starting step
scenes_valid = env.reset(scene_indices=scene_indices, start_frame_index=None)
scene_indices = [si for si, sval in zip(scene_indices, scenes_valid) if sval]

# if requested, split each scene up into multiple simulations
print(f'exp_config.algo.history_num_frames: {exp_config.algo.history_num_frames}')
start_frame_index = [[exp_config.algo.history_num_frames+1]] * len(scene_indices)

# num_sim_per_scene is predefined in the config.json. I think it's 1 for all the agent
# how many sims to run for the current batch of scenes
print('Starting frames in current scenes:')
print(start_frame_index)

num_scenes_per_batch: 1
[0]
exp_config.algo.history_num_frames: 30
Starting frames in current scenes:
[[31]]


In [74]:
# during each sim in the scene
# for ei in range(eval_cfg.num_sim_per_scene) -- it's 1 for all scene so we just skip this
ei = 0
guidance_config = None # for the current batch of scenes

cur_start_frames = [scene_start[ei] for scene_start in start_frame_index]
# double check all scenes are valid at the current start step
scenes_valid = env.reset(scene_indices=scene_indices, start_frame_index=cur_start_frames)
sim_scene_indices = [si for si, sval in zip(scene_indices, scenes_valid) if sval]
sim_start_frames = [sframe for sframe, sval in zip(cur_start_frames, scenes_valid) if sval]


In [85]:
# build heuristic_guidance -- is this where i should make modification?
if "config" in eval_cfg.edits.editing_source:
    guidance_config = eval_cfg.edits.guidance_config
if "heuristic" in eval_cfg.edits.editing_source:
    # build heuristic guidance configs for these scenes
    from tbsim.utils.scene_edit_utils import compute_heuristic_guidance, merge_guidance_configs
    heuristic_guidance_cfg = compute_heuristic_guidance(heuristic_config,
                                                        env,
                                                        sim_scene_indices,
                                                        sim_start_frames)
print(heuristic_config)
print(heuristic_guidance_cfg)

# check if the guidance is available
if len(heuristic_config) > 0:
    valid_scene_inds = []
    for sci, sc_cfg in enumerate(heuristic_guidance_cfg):
        if len(sc_cfg) > 0:
            valid_scene_inds.append(sci)
    # collect only valid scenes under the given heuristic config
    heuristic_guidance_cfg = [heuristic_guidance_cfg[vi] for vi in valid_scene_inds]
    sim_scene_indices = [sim_scene_indices[vi] for vi in valid_scene_inds]
    sim_start_frames = [sim_start_frames[vi] for vi in valid_scene_inds]
    # skip if no valid
# add to the current guidance config
guidance_config = merge_guidance_configs(guidance_config, heuristic_guidance_cfg)

[{'name': 'target_pos', 'weight': 30000.0, 'params': {'target_time': 40}}]
[[{'name': 'target_pos', 'params': {'target_pos': [[5.590123545755198, -1.174515570518878], [5.76083234951316, -1.2723765352821559], [4.147990116712341, -2.2647575142523846], [5.003380129924496, -1.6954651989966685], [3.8003058803357117, -0.8746731123194786], [0.2061683719248446, 4.200072529769727e-07], [7.353745123561403, -0.5799873588514646], [5.801846577075657, 0.017430045198318034]]}, 'agents': [0, 1, 2, 3, 4, 5, 6, 7], 'weight': 30000.0}]]


## guided_rollout
stats, info = guided_rollout( \
    env, \
    rollout_policy, \
    policy_model, \
    n_step_action=eval_cfg.n_step_action, \
    guidance_config=guidance_config, \
    scene_indices=sim_scene_indices, \
    obs_to_torch=True, \
    horizon=eval_cfg.num_simulation_steps, \
    use_gt=use_gt, \
    start_frames=sim_start_frames, \
)  

In [87]:
# init
obs_to_torch = True
policy = rollout_policy
from tbsim.envs.base import BatchedEnv
import tbsim.utils.tensor_utils as TensorUtils

stats = {}
info = {}
is_batched_env = isinstance(env, BatchedEnv)

In [109]:
# setup guidance and associated metrics
added_metrics = []
if guidance_config is not None:
    env.reset(scene_indices=sim_scene_indices, start_frame_index=sim_start_frames)
    ex_obs = env.get_observation()
    if obs_to_torch:
        device = policy.device if device is None else device
        ex_obs = TensorUtils.to_torch(ex_obs, device=device, ignore_if_unspecified=True)
    if not use_gt:
        # polciy_model - DiffuserTrafficModel
        # policy_model.set_guidance -> cur_policy.set_guidance() -> self.nets["policy"].set_guidance() -> DiffuserModel.
        # self.current_guidance = DiffuserGuidance(guidance_config_list, example_batch)
        # example_batch= ex_obs['agents'] - obs
        # set_guidance is set the GuidanceLoss for each type
        policy_model.set_guidance(guidance_config, ex_obs['agents'])
    from tbsim.utils.guidance_metrics import guidance_metrics_from_config
    # guidance_name_to_metrics
    guidance_metrics = guidance_metrics_from_config(guidance_config)
    env._metrics.update(guidance_metrics) 
    added_metrics += guidance_metrics.keys()
# metrics are reset here too, so have to run again after adding new metrics
env.reset(scene_indices=sim_scene_indices, start_frame_index=sim_start_frames)

Instantiating test-time guidance with configs:
[[{'name': 'target_pos', 'params': {'target_pos': [[5.590123545755198, -1.174515570518878], [5.76083234951316, -1.2723765352821559], [4.147990116712341, -2.2647575142523846], [5.003380129924496, -1.6954651989966685], [3.8003058803357117, -0.8746731123194786], [0.2061683719248446, 4.200072529769727e-07], [7.353745123561403, -0.5799873588514646], [5.801846577075657, 0.017430045198318034]]}, 'agents': [0, 1, 2, 3, 4, 5, 6, 7], 'weight': 30000.0}]]


[True]

### start of real rollout

In [111]:
done = env.is_done()
counter = 0
# while not done:
# let's just try the first step
obs = env.get_observation()
if obs_to_torch:
    device = policy.device if device is None else device
    obs_torch = TensorUtils.to_torch(obs, device=device, ignore_if_unspecified=True)
else:
    obs_torch = obs
action = policy.get_action(obs_torch, step_index=counter)

check if dynamics type is not none
> /local-scratch2/lufan/traceMR/tbsim/models/trace.py(257)get_aux_info()
    255             print('check if dynamics type is not none')
    256             import pdb; pdb.set_trace()
--> 257             curr_states = batch_utils().get_current_states(data_batch, dyn_type=self.dyn.type())
    258         else:
    259             curr_states = None



/localhome/lya108/miniconda3/envs/tracer/lib/python3.8/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/localhome/lya108/miniconda3/envs/tracer/lib/python3.8/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


===== GUIDANCE LOSSES ======
target_pos_scene_000_00: 0.049981225282


In [114]:
dir(action)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'agents',
 'agents_info',
 'ego',
 'ego_info',
 'from_dict',
 'has_agents',
 'has_ego',
 'to_dict',
 'to_numpy',
 'transform']

In [120]:
action.agents_info.keys()

dict_keys(['action_samples', 'diffusion_steps'])

In [127]:
action.agents_info['action_samples']['positions'].shape

torch.Size([8, 20, 52, 2])

In [128]:
action.agents_info['action_samples']['yaws'].shape

torch.Size([8, 20, 52, 1])

In [123]:
action.agents_info['diffusion_steps']['traj'].shape

torch.Size([8, 52, 101, 6])

In [108]:
guidance_metrics

{'guide_target_pos_s0g0': <tbsim.utils.guidance_metrics.TargetPosGuidance at 0x72726dd02bb0>}

In [101]:
guidance_config

[[{'name': 'target_pos',
   'params': {'target_pos': [[5.590123545755198, -1.174515570518878],
     [5.76083234951316, -1.2723765352821559],
     [4.147990116712341, -2.2647575142523846],
     [5.003380129924496, -1.6954651989966685],
     [3.8003058803357117, -0.8746731123194786],
     [0.2061683719248446, 4.200072529769727e-07],
     [7.353745123561403, -0.5799873588514646],
     [5.801846577075657, 0.017430045198318034]]},
   'agents': [0, 1, 2, 3, 4, 5, 6, 7],
   'weight': 30000.0}]]

In [23]:
cur_scene = env._current_scenes[0]

In [4]:
import json
import os

# Read collision statistics from stats.json
cfg_l = ['tg', 'agent_coll', 'agent_coll_tg' 'mapf_tg_w500', 'mapf_tg_w200']
path_l = [f'orca_mixed_out_{config}' for config in cfg_l]
all_coll_rate = {}
disk_coll_rate = {}
agent_coll_rate = {}

for path in path_l:
    stats_path = f"/local-scratch2/lufan/traceMR/out/{path}/orca_map_open_loop_agent_coll/stats.json"
    if os.path.exists(stats_path):
        with open(stats_path, 'r') as f:
            stats = json.load(f)
            
        # Extract collision rates
        cur_all_coll_rate = stats.get('all_collision_rate_coll_any', None)
        cur_disk_coll_rate = stats.get('all_disk_collision_rate_coll_any', None) 
        cur_agent_coll_rate = stats.get('agents_collision_rate_coll_any', None)
        all_coll_rate[path] = cur_all_coll_rate
        disk_coll_rate[path] = cur_disk_coll_rate
        agent_coll_rate[path] = cur_agent_coll_rate
    else:
        print(f"Stats file not found at {stats_path}")


Stats file not found at /local-scratch2/lufan/traceMR//out/orca_mixed_out_tg/orca_map_open_loop_agent_coll/stats.json
Stats file not found at /local-scratch2/lufan/traceMR//out/orca_mixed_out_agent_coll_tgmapf_tg_w500/orca_map_open_loop_agent_coll/stats.json
Stats file not found at /local-scratch2/lufan/traceMR//out/orca_mixed_out_mapf_tg_w200/orca_map_open_loop_agent_coll/stats.json


# Result

In [45]:
# baselines
# 1. target pos only
path = '/local-scratch2/lufan/traceMR/out/orca_mixed_out/orca_map_open_loop_target_pos/stats.json'
if os.path.exists(path):
        with open(path, 'r') as f:
            stats = json.load(f)
        all_coll_rate = stats.get('all_collision_rate_coll_any', None)
        disk_coll_rate = stats.get('all_disk_collision_rate_coll_any', None) 
        agent_coll_rate = stats.get('agents_collision_rate_coll_any', None)
        scene_idx = stats.get('scene_index', None)
else:
    print(f"Stats file not found at {path}")
a = np.array(all_coll_rate)
mean = a.mean()
std = a.std()
max = a.max()
print(mean)
print(std)
print(max)
idx_worst_3 = np.argsort(a)[-3:]
worst_3_values = a[idx_worst_3]


0.162281746031746
0.2111314688761148
0.75


In [46]:
for idx in idx_worst_3:
    scene = scene_idx[idx]
    print(scene)

scene_000125_orca_maps
scene_000682_orca_maps
scene_000202_orca_maps


In [39]:
worst_3_values

array([0.66666667, 0.75      , 0.75      ])

In [47]:
# 2. target pos + agent collision
path = '/local-scratch2/lufan/traceMR/out/orca_mixed_out_agent_coll_tg/orca_map_open_loop_agent_coll/stats.json'
if os.path.exists(path):
        with open(path, 'r') as f:
            stats = json.load(f)
        all_coll_rate = stats.get('all_collision_rate_coll_any', None)
        disk_coll_rate = stats.get('all_disk_collision_rate_coll_any', None) 
        agent_coll_rate = stats.get('agents_collision_rate_coll_any', None)
        scene_idx = stats.get('scene_index', None)
else:
    print(f"Stats file not found at {path}")

a = np.array(all_coll_rate)
mean = a.mean()
std = a.std()
max = a.max()
print(mean)
print(std)
print(max)
idx_worst_3 = np.argsort(a)[-3:]
worst_3_values = a[idx_worst_3]

for idx in idx_worst_3:
    scene = scene_idx[idx]
    print(scene)


0.05932936507936508
0.14179997390522078
0.6666666666666666
scene_000447_orca_maps
scene_000617_orca_maps
scene_000125_orca_maps


In [61]:
import torch
a = torch.randn((2, 4))
b = torch.randint(low=0, high=4, size=(2,))
c = a.gather(1, b.unsqueeze(-1))

In [62]:
b

tensor([3, 0])

In [63]:
a

tensor([[-1.2446,  0.5071,  0.6448, -0.6639],
        [-1.9677, -0.6715,  1.3465,  0.2245]])

In [67]:
c.squeeze(-1).shape

torch.Size([2])

TypeError: gather() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (int dim, Tensor index, *, bool sparse_grad)
 * (name dim, Tensor index, *, bool sparse_grad)


In [48]:
worst_3_values

array([0.5       , 0.5       , 0.66666667])

In [36]:
import numpy as np
import os

path = '/local-scratch2/lufan/traceMR/out/orca_mixed_out_mapf_tg_w500/orca_map_open_loop_target_pos_all_coll/stats.json'
if os.path.exists(path):
        with open(path, 'r') as f:
            stats = json.load(f)
        all_coll_rate = stats.get('all_collision_rate_coll_any', None)
        disk_coll_rate = stats.get('all_disk_collision_rate_coll_any', None) 
        agent_coll_rate = stats.get('agents_collision_rate_coll_any', None)
else:
    print(f"Stats file not found at {path}")

a = np.array(all_coll_rate)
mean = a.mean()
std = a.std()
max = a.max()
print(mean)
print(std)
print(max)
idx_worst_3 = np.argsort(a)[-3:]
worst_3_values = a[idx_worst_3]

0.28199206349206346
0.3025590535987777
1.0


In [37]:
worst_3_values

array([1., 1., 1.])

In [31]:
a = np.array(all_coll_rate)
mean = a.mean()
std = a.std()
max = a.max()
print(mean)
print(std)
print(max)

0.05932936507936508
0.14179997390522078
0.6666666666666666


In [33]:
scene_idx = stats.get('scene_index', None)

In [34]:
scene_idx[12]

'scene_000125_orca_maps'

In [35]:
scene_idx[21]

'scene_000202_orca_maps'

In [68]:
# 2. target pos + agent collision
path = '/local-scratch2/lufan/traceMR/out/orca_mixed_out_mapf_tg_w500/orca_map_open_loop_target_pos_all_coll/stats.json'
if os.path.exists(path):
        with open(path, 'r') as f:
            stats = json.load(f)
        all_coll_rate = stats.get('all_collision_rate_coll_any', None)
        disk_coll_rate = stats.get('all_disk_collision_rate_coll_any', None) 
        agent_coll_rate = stats.get('agents_collision_rate_coll_any', None)
        scene_idx = stats.get('scene_index', None)
else:
    print(f"Stats file not found at {path}")

a = np.array(all_coll_rate)
mean = a.mean()
std = a.std()
max = a.max()
print(mean)
print(std)
print(max)
idx_worst_3 = np.argsort(a)[-3:]
worst_3_values = a[idx_worst_3]

for idx in idx_worst_3:
    scene = scene_idx[idx]
    print(scene)


0.24240079365079364
0.2951144741135509
1.0
scene_000662_orca_maps
scene_000881_orca_maps
scene_000801_orca_maps
